<a href="https://colab.research.google.com/github/LillianLimd/openFrameworks/blob/master/GPU_Style_Transfer_Training_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What are we lookin' at here?

![Style Transfer Reference](https://drive.google.com/uc?id=1YV2CtutL2bMX5sVc8zxwCm4WqWNJdmSA)

**Colab Notebooks**:

This is a Google Colab Jupyter Notebook. If you've never used one before, they're a cool way to run code without having to do a bunch of setup on your own computer. You run the code by pushing the play button next to each code block below.

**Style Transfer**: 

This particular notebook shows you how to train a Style Transfer Machine Learning model on your own images. Style Transfer takes the content of one image (hereafter referred to as the `content image`) and renders them in the style of another image (the `style image`).

# Preparation

**Before you do anything in this notebook:**

In a separate browser tab, open up Google drive. 

Make a folder called `colab-files` and put a content image (item you want to stylize) in your Google drive, as well as a style image (style of the image you want to mimic). For this demo, we'll put them in the `colab-files` folder we just made, and call them `style.jpg` and `content.jpg`. These should be small (max 600px w h ) to eliminate longer training times than colab will allow.

In that same folder, put two more folders called `checkpoint` and `test`

---



# Mounting Your Drive

Click play below, authenticate and allow Google to mount your drive, then copy the link and paste in the field below, then hit return to mount your google drive within this notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

list the files at the root of your google drive, and make sure you can see your `colab-files` directory.

In [ ]:
!ls "/content/drive/My Drive/colab-files"

# Runtime Hardware

Make sure you're using Colab's GPUs. You can enable them by going to `Runtime > Change Runtime Type` and Select `Runtime Type: Python 3, Hardware Accelerator: GPU, and Runtime Shape: High RAM`

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print('you are using the GPU!')
  print(gpu_info)

Make sure you're using available RAM

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

# Get Fast Style Transfer Repo

In [ ]:
!git clone https://github.com/lengstrom/fast-style-transfer

Change to the fast-style-transfer directory

In [ ]:
cd '/content/fast-style-transfer'

# Setup

Run the setup script, which will download the necessary dependencies, including a very large dataset - this has taken me as little as 10 and as much as about 20 minutes. It's done when the spinning stop icon goes away.

---



In [ ]:
!sh setup.sh

Fix an issue with incompatibilities in the latest scipy by installing a specific, older version. When it prompts you to proceed, type `y` then hit enter.

In [ ]:
!pip uninstall scipy
!pip install scipy==1.1.0

# Training

We can now begin training.

Make sure that if you created your style and content images or your checkpoint and test directories in a different google drive location, you change the paths below.

You can adjust the content weight here. Higher numbers favor maintaining the content image's appearance, and lower favor the style image.

Change the iterations to whatever you'd like. The default for this repo is 2000. 

Begin training by hitting play below. **This will take a long time** (~6 hours for defaults with a small image). You can speed it up by lowering the number of iterations. If you just want to get a sense of how things look, try 100 iterations.

Note that Google Colab can disconnect you, and those disconnect times are uncertain, but [seem to max out at about 12 hours of GPU usage](https://stackoverflow.com/questions/55050988/can-i-run-a-google-colab-free-edition-script-and-then-shutdown-my-computer).



In [ ]:
!python style.py --style /content/drive/My\ Drive/colab-files/style.jpg \
  --checkpoint-dir /content/drive/My\ Drive/colab-files/checkpoint \
  --test /content/drive/My\ Drive/colab-files/content.jpg \
  --test-dir /content/drive/My\ Drive/colab-files/test \
  --content-weight 15 \
  --checkpoint-iterations 900 \
  --batch-size 20

**Q**: Is it working? This is taking forever...

**A**:

Is the play button a stop icon? Is it spinning?

Do you see something in the output like:

`Epoch 0, Iteration: 1300, Loss: 13707578.0`

Are there multiple of these?
Are the iterations increasing?
Is this loss generally going down?

If you look in your `checkpoint` directory in your google drive, do you see a few files, including

```
checkpoint
fns.ckpt.index
fns.ckpt.meta
fns.ckpt.data-...
```

Is that meta file more than 100MB in size?

If all of these are true - yes, it is probably working! and learning! and saving!



---



**Q**: Can I Stop It?

**A**: Yes, you should be able to stop the training and use the results of your checkpoint directory in for the steps below.

---

**Q**: What now?

**A**: You can use this model in Tensorflow by downloading the "checkpoint" directory. 

**If you want to use this model in Tensorflow.js**: read on for how to convert it tensorflow.js format and run a visualizer which renders your webcam feed in the style of your trained image.

---

**Q**: What's up with that `test` directory?

**A**: It should be full of images. That's the result of your model as it trains. You should see the images getting closer and closer to a blend of the content and style image as they train.

![Training Process](https://drive.google.com/uc?id=1FASMK9C59vBOH0EQhCZE3E-BNhaMOYiB)

---




# Applying Style Transfer to Multiple Images

If you want to use your model on many content images at once, gather those images, place them in a folder called `evaluate` within your `colab-files` directory on Google Drive, make another empty folder called `results` within the `colab-files` directory, and then run the evaluate script:

In [ ]:
!python evaluate.py --checkpoint /content/drive/My\ Drive/colab-files/checkpoint/ \
  --in-path /content/drive/My\ Drive/colab-files/evaluate/ \
  --out-path /content/drive/My\ Drive/colab-files/results/ \
  --allow-different-dimensions

# Converting to Tensorflow.js

The following repo has a script that will help us convert our files from tensorflow to tensorflow.js-friendly format. We're going to clone it - the default folder I'm specifying will be the `colab-files/fst` directory.

In [ ]:
!git clone https://github.com/reiinakano/fast-style-transfer-deeplearnjs.git /content/drive/My\ Drive/colab-files/fst

...and now we change into that directory...

In [ ]:
cd /content/drive/My\ Drive/colab-files/fst

In [ ]:
!python scripts/dump_checkpoint_vars.py --output_dir=src/ckpts/mystyle --checkpoint_file=/content/drive/My\ Drive/colab-files/checkpoint/fns.ckpt

In [ ]:
!python scripts/remove_optimizer_variables.py --output_dir=src/ckpts/mystyle

You are done! Go to the directory on your Google Drive where these have been saved (`colab-files/src/checkpoints/mystyle` if you didn't change anything) and download the `mystyle` folder that was made.

You can now use this within tensorflow.js. Follow along below for an example of how to do that with a javascript library called ML5 and your webcam.

---

# Webcam Style Transfer Visualizer in the Browser Using ML5 and Tensorflow.js

If you want to see your results, clone or download [this repo](https://github.com/heaversm/colab-style-transfer-training-test) and follow the instructions in the README.md file in order to visualize the content of your webcam in the style of your newly trained model!

This notebook was only possible through the code and documentation of [Logan Engstrom et al](https://github.com/lengstrom/fast-style-transfer), [Reii Nakano](https://github.com/reiinakano), and [Yining Shi](https://github.com/yining1023).



---

# Thanks!

If any of this was helpful, I always love to hear about it and see what people create!